In [24]:
#Import the Spotipy library, which is a Python library for the Spotify Web API.
import spotipy
#Import SpotifyClientCredentials from spotipy.oauth2 module, which provides client credentials flow for Spotify API authentication.
from spotipy.oauth2 import SpotifyClientCredentials
from datetime import datetime  
import pandas as pd
#Import the sqlalchemy library, which is used for interacting with relational databases using SQL.
import sqlalchemy as sal

In [25]:
# Spotify API credentials
spotify_credentials = SpotifyClientCredentials('b353b56451d84f2abd7945c34b97b3fb', '01ca601fda9f4f64a618ebaef441f187')
spotify = spotipy.Spotify(client_credentials_manager=spotify_credentials)

In [26]:
# Connect to SQL Server
engine = sal.create_engine('mssql://LAPTOP-27O058ES\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [4]:
#https://open.spotify.com/playlist/4hOKQuZbraPDIfaGbM3lKI
tracks = spotify.playlist_tracks('4hOKQuZbraPDIfaGbM3lKI')

In [5]:
tracks.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [6]:
type(tracks['items'][0].keys())

dict_keys

In [7]:
track=tracks['items'][0]
track['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [34]:
track['track']['duration_ms']/1000

160.596

In [28]:
song_data = []
for track in tracks['items']:
    track_info = {
        'Title': track['track']['name'],
        'Artist': track['track']['artists'][0]['name'],
        'Album': track['track']['album']['name'],
        'ReleaseDate': track['track']['album']['release_date'],
        'Popularity': track['track']['popularity'],
        'Duration': track['track']['duration_ms']/1000,
        'SnapshotDate': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    song_data.append(track_info)

# Create a DataFrame using pandas
df = pd.DataFrame(song_data)
df

,Title,Artist,Album,ReleaseDate,Popularity,Duration,SnapshotDate
0,Despacito - Remix,Luis Fonsi,Despacito Feat. Justin Bieber (Remix),2017-04-17,2,228.826,2024-01-21 18:33:05
1,I'm the One,DJ Khaled,I'm the One,2017-04-27,0,288.876,2024-01-21 18:33:05
2,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,89,233.712,2024-01-21 18:33:05
3,HUMBLE.,Kendrick Lamar,DAMN.,2017-04-14,87,177.000,2024-01-21 18:33:05
4,Mask Off,Future,FUTURE,2017-08-04,51,204.600,2024-01-21 18:33:05
...,...,...,...,...,...,...,...
95,24K Magic,Bruno Mars,24K Magic,2016-11-17,83,225.983,2024-01-21 18:33:05
96,Tu Foto,Ozuna,Tu Foto,2017-04-27,0,193.117,2024-01-21 18:33:05
97,T-Shirt,Migos,Culture,2017-04-07,1,242.407,2024-01-21 18:33:05
98,Sola (Remix),Anuel AA,Sola (Remix),2016-12-01,78,307.910,2024-01-21 18:33:05


In [35]:
# Write the contents of the DataFrame 'df' to a SQL table named 'SpotifyData'.
# 'con' is the SQLAlchemy connection object, and 'index=False' means not to include the DataFrame index as a separate column.
df.to_sql('SpotifyData',con=conn , index=False , if_exists = 'append')
conn.commit()